В работе используется модель FRED-T5

# Загрузка датасета

In [ ]:
!git clone https://github.com/awe-f0x/LLM-for-business-content-creation.git
DATA_PATH = "LLM-for-business-content-creation/data/dataset.json"

import json
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Всего текстов: {len(data)}")

fatal: destination path 'LLM-for-business-content-creation' already exists and is not an empty directory.
Всего текстов: 1040


In [ ]:
sample_key = next(iter(data))
sample_text = data[sample_key]['text']

print("=== Оригинальный текст ===")
print(sample_text[:300] + "...")
print(f"\nДлина: {len(sample_text.split())} слов")

=== Оригинальный текст ===
Прибыль сектора сократилась
до 770 млрд руб. 1 (-14% 
по сравнению с 1к24), 
в основном из-за роста операционных расходов, снижения доходов от инвалюты, а также убытка от ценных бумаг (повлияла отрицательная переоценка ОФЗ вследствие ожиданий по ужесточению ДКУ).
Рост ипотеки ускорился 
до 6,3% 2 (2...

Длина: 309 слов


# Проверка данных

In [ ]:
print(f"Тип данных: {type(data)}")

if isinstance(data, list):
    print(f"Всего текстов: {len(data)}")
    if len(data) > 0:
        print("\nПример текста:")
        print(json.dumps(data[0], ensure_ascii=False, indent=2))
elif isinstance(data, dict):
    print("\nДанные в формате словаря. Доступные ключи:")
    print(list(data.keys()))
    # Выведем первый элемент, если есть вложенные данные
    first_key = next(iter(data))
    print(f"\nПример данных по ключу '{first_key}':")
    print(json.dumps(data[first_key], ensure_ascii=False, indent=2))
else:
    print("Неизвестный формат данных.")

Тип данных: <class 'dict'>

Данные в формате словаря. Доступные ключи:
['8d278789-c952-4ed6-ad5d-65e0859edafa', 'a03c5fbe-da58-4c51-9120-76e0169c270d', 'aec2d27b-ca54-48fc-bd09-7e6f2527ed39', '308f4c83-b52c-4d3e-bc84-110ce02ee060', '08132b18-4235-4b94-be59-670bd05928a3', '55a0e018-345e-4a5e-8f37-cf79cb270e5c', '46d7df7e-7a8e-40ec-b5e8-093a5904a664', 'db336eb5-77cd-418f-b27d-0ba3bf39306e', 'a84b82f5-963f-4688-b330-6b85d690a6b5', '474ba176-b460-4997-9800-c3a8160c85a5', 'cf728d14-9731-451e-8c6c-d273a88cff27', 'b4333e8a-fc79-4ccb-9b2d-4113e95ba04f', 'f6171eb2-db0e-4027-8a95-da4819c972a7', 'aa9097b8-bd0a-459f-ae19-2eb569276717', '400ae9cf-3803-40ae-93f5-f3a0554f000b', '434ed983-6dda-4c10-841e-977d2a59f20e', '71434e9e-2589-4dad-bcb1-294c4159a153', '74e79c20-6c69-4d2a-a732-6e74b8b08138', '7950675f-2376-4003-b59a-edf9ba3ed3f7', '79eadaf2-91c3-4e30-8a87-db6b03f8f86b', '2c39ea7d-1083-455d-b968-1af5142904f4', '12e601ed-1219-4cbd-a0aa-c1c0426340f5', 'ff652931-49a5-4d2d-9991-af00b5ad61b1', '48e15be

# Установка зависимостей

In [ ]:
!pip install transformers sentencepiece pymorphy3 syllables

# Импорт библиотек и инициализация

In [ ]:
import re
import pymorphy3
import syllables
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Инициализация анализатора
morph = pymorphy3.MorphAnalyzer()

# Загрузка модели
model_name = "cointegrated/rut5-base-absum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Метрика AlinaEstimator

In [ ]:
class AlinaEstimator:
    @staticmethod
    def calculate(text):
        """Оценка прозрачности 0-10 на основе:
        - Лексики, морфологии, синтаксиса"""
        words = re.findall(r'\w+', text.lower())
        if not words:
            return 0.0

        # Лексическая сложность
        unique_ratio = len(set(words)) / len(words)

        # Морфологическая сложность
        long_words = sum(1 for w in words if len(w) > 8) / len(words)

        # Синтаксическая сложность
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]
        avg_sent_len = sum(len(s.split()) for s in sentences) / len(sentences) if sentences else 0

        # Итоговая формула
        score = 10 * (1 - 0.4*unique_ratio - 0.4*long_words - 0.2*(avg_sent_len/20))
        return max(0, min(round(score, 1), 10))

# Метрика FRE

In [ ]:
class FRECalculator:
    @staticmethod
    def calculate(text):
        """Реализация Flesch Reading Ease для русского языка"""
        words = re.findall(r'\w+', text)
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]

        if not words or not sentences:
            return 0.0

        # Подсчет слогов
        syllable_count = sum(syllables.estimate(w) for w in words)

        fre = 206.835 - (1.015 * (len(words)/len(sentences))) - (84.6 * (syllable_count/len(words)))
        return max(0, min(round(fre, 1), 100))

# Ищем термины

In [ ]:
class TermDetector:
    def __init__(self):
        self.common_words = {
            'финансовый', 'банковский', 'кредитный',
            'регулирование', 'система', 'рынок'
        }

    def find_terms(self, text):
        words = re.findall(r'\b[а-яё]{8,}\b', text.lower())
        abbreviations = re.findall(r'\b[А-Я]{2,}\b', text)

        terms = []
        for word in set(words + abbreviations):
            parsed = morph.parse(word)[0]
            if parsed.normal_form not in self.common_words:
                terms.append(word)
        return terms

# Упрощаем текст

In [ ]:
class TextSimplifier:
    def __init__(self):
        self.term_detector = TermDetector()

    def simplify(self, text):
        terms = self.term_detector.find_terms(text)
        print(f"Найдены термины: {terms}")

        for term in terms:
            if re.fullmatch(r'[А-Я]{2,}', term):
                text = text.replace(term, f"{term} (расшифровка)")
            else:
                root = morph.parse(term)[0].normal_form[:6] + ".."
                text = text.replace(term, root)

        text = self._simplify_syntax(text)
        return text

    def _simplify_syntax(self, text):
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]
        simplified = []

        for sent in sentences:
            if len(sent.split()) > 15:
                parts = re.split(r'[,;] (?:и|а|но|что|которые) ', sent)
                simplified.extend(p for p in parts if p)
            else:
                simplified.append(sent)

        result = []
        for sent in simplified:
            sent = re.sub(r'был[ао]?\s(\w+ен\w*)', r'\1или', sent)  # Пассив в актив
            sent = re.sub(r'\bосуществляется\b', 'проводится', sent)
            result.append(sent)

        return ' '.join(result)

# LLM

In [ ]:
class TextRefiner:
    def refine(self, text):
        inputs = tokenizer(
            f"Упрости текст, сохраняя суть: {text}",
            return_tensors="pt",
            max_length=512,
            truncation=True
        )

        outputs = model.generate(
            inputs.input_ids,
            max_length=256,
            num_beams=3,
            early_stopping=True
        )

        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
class Pipeline:
    def __init__(self):
        self.simplifier = TextSimplifier()
        self.refiner = TextRefiner()
        self.alina = AlinaEstimator()
        self.fre = FRECalculator()

    def process(self, text):
        print("=== Исходный текст ===\n", text[:200] + "...\n")

        # Оценка качества
        orig_metrics = {
            'Alina': self.alina.calculate(text),
            'FRE': self.fre.calculate(text)
        }


        simplified = self.simplifier.simplify(text)
        refined = self.refiner.refine(simplified)


        refined_metrics = {
            'Alina': self.alina.calculate(refined),
            'FRE': self.fre.calculate(refined)
        }

        return {
            'text': refined,
            'metrics': {
                'original': orig_metrics,
                'simplified': refined_metrics
            }
        }

# Пример использования

In [ ]:
sample_text = (
    "Прибыль сектора сократилась  до 770 млрд руб. 1 (-14% по сравнению с 1к24), в основном из-за роста операционных расходов, снижения доходов от инвалюты, а также убытка от ценных бумаг (повлияла отрицательная переоценка ОФЗ вследствие ожиданий по ужесточению ДКУ)."
)


pipeline = Pipeline()
result = pipeline.process(sample_text)

print("\n=== Результат ===")
print(result['text'])
print("\n=== Метрики ===")
print(f"Alina: {result['metrics']['original']['Alina']} → {result['metrics']['simplified']['Alina']}")
print(f"FRE: {result['metrics']['original']['FRE']} → {result['metrics']['simplified']['FRE']}")

=== Исходный текст ===
 Прибыль сектора сократилась  до 770 млрд руб. 1 (-14% по сравнению с 1к24), в основном из-за роста операционных расходов, снижения доходов от инвалюты, а также убытка от ценных бумаг (повлияла отрицат...

Найдены термины: ['переоценка', 'ОФЗ', 'инвалюты', 'снижения', 'основном', 'сравнению', 'операционных', 'повлияла', 'отрицательная', 'сократилась', 'ужесточению', 'вследствие', 'расходов', 'ДКУ', 'ожиданий']

=== Результат ===
По итогам прошлого года экономический рост сектора сократился до 770 млрд руб.

=== Метрики ===
Alina: 3.6 → 4.1
FRE: 100 → 100
